In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import OpenAIGPTTokenizer, OpenAIGPTLMHeadModel, AdamW, get_linear_schedule_with_warmup, GPT2Tokenizer, OPTForCausalLM
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv, os

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Data Preparation

In [4]:
# df_train = pd.read_csv("../Data/Actual Data/train_data_pt.csv")
df_train = pd.read_csv("../Data/Actual Data/one-shot-train_data_pt.csv")

df_train.head()

,Unnamed: 0,MWE,Previous,Target,Next,Label
0,87,segundo plano,"Para tanto, basta ter as versões mais recentes...","A partir daí, a tecnologia do Face ID fica em ...","Para desbloquear o celular usando máscara, o A...",0
1,88,desfile militar,"Lutaremos até que a junta militar caia”, acres...",Os ataques das forças de segurança ocorreram s...,Atos violentos que afetam a segurança e a esta...,1
2,89,sangue quente,"Na cobrança, Chiquinho pegou firme e venceu Poli.","Como em todo clássico, o sangue quente apareceu.","Primeiro, Marcel soltou a mão no rosto de Thye...",0
3,90,sangue quente,Ambos os animais tinham menos de trinta centím...,Os tritilodontídeos em geral eram semelhantes ...,“Cada um dos dois representa a única espécie e...,1
4,91,alto-falante,Adquiria o Beosound Emerge no site oficial Ban...,"Segundo a empresa, a intenção foi construir um...",O design permite colocar o Beosound Emerge no ...,1


In [5]:
# Previous + Target
df_train['Context'] = df_train['Previous'] + ' ' + df_train['Target']
df_train['All'] = df_train['Previous'] + ' ' + df_train['Target'] + ' ' + df_train['Next']
df_train.head()

,Unnamed: 0,MWE,Previous,Target,Next,Label,Context,All
0,87,segundo plano,"Para tanto, basta ter as versões mais recentes...","A partir daí, a tecnologia do Face ID fica em ...","Para desbloquear o celular usando máscara, o A...",0,"Para tanto, basta ter as versões mais recentes...","Para tanto, basta ter as versões mais recentes..."
1,88,desfile militar,"Lutaremos até que a junta militar caia”, acres...",Os ataques das forças de segurança ocorreram s...,Atos violentos que afetam a segurança e a esta...,1,"Lutaremos até que a junta militar caia”, acres...","Lutaremos até que a junta militar caia”, acres..."
2,89,sangue quente,"Na cobrança, Chiquinho pegou firme e venceu Poli.","Como em todo clássico, o sangue quente apareceu.","Primeiro, Marcel soltou a mão no rosto de Thye...",0,"Na cobrança, Chiquinho pegou firme e venceu Po...","Na cobrança, Chiquinho pegou firme e venceu Po..."
3,90,sangue quente,Ambos os animais tinham menos de trinta centím...,Os tritilodontídeos em geral eram semelhantes ...,“Cada um dos dois representa a única espécie e...,1,Ambos os animais tinham menos de trinta centím...,Ambos os animais tinham menos de trinta centím...
4,91,alto-falante,Adquiria o Beosound Emerge no site oficial Ban...,"Segundo a empresa, a intenção foi construir um...",O design permite colocar o Beosound Emerge no ...,1,Adquiria o Beosound Emerge no site oficial Ban...,Adquiria o Beosound Emerge no site oficial Ban...


In [6]:
df_test = pd.read_csv("../Data/Actual Data/test_data_pt.csv")
df_test.Label.value_counts()

0    154
1    119
Name: Label, dtype: int64

In [7]:
df_test = df_test.groupby('Label').sample(n=119, random_state=42)
# df_test = df_test[['MWE', 'Previous', 'Next', 'Target', 'Label']].copy()
df_test.Label.value_counts()

0    119
1    119
Name: Label, dtype: int64

In [8]:
# Previous + Target
df_test['Context'] = df_test['Previous'] + ' ' + df_test['Target']
df_test['All'] = df_test['Previous'] + ' ' + df_test['Target'] + ' ' + df_test['Next']
df_test.head()

,Unnamed: 0,MWE,Previous,Target,Next,Label,Context,All
31,497,sangue quente,E é claro aproveitou a parada para conversar e...,"Após o jogo, o português de uma pequena amostr...",Entrou em campo para reclamar e cobrar o árbit...,0,E é claro aproveitou a parada para conversar e...,E é claro aproveitou a parada para conversar e...
145,611,buraco negro,O achado foi publicado nesta quarta-feira (21)...,Os autores da pesquisa apelidaram o buraco neg...,Anel de poeira na órbita de Vênus é visto por ...,0,O achado foi publicado nesta quarta-feira (21)...,O achado foi publicado nesta quarta-feira (21)...
257,723,colégio militar,O prefeito estava acompanhado dos secretários ...,“O Colégio Militar tem uma estrutura diferenci...,"Os estudantes, que ainda estão assistindo às a...",0,O prefeito estava acompanhado dos secretários ...,O prefeito estava acompanhado dos secretários ...
156,622,buraco negro,"O filme tem o protagonismo de Antônio Pitanga,...",Isto o guiará num buraco negro profundo e comp...,A inclusão do filme brasileiro na seleção ofic...,0,"O filme tem o protagonismo de Antônio Pitanga,...","O filme tem o protagonismo de Antônio Pitanga,..."
200,666,bode expiatório,Exclusivo Treinador dos sub-23 do Leixões como...,Vale a pena transformar José Augusto Faria no ...,Vale.,0,Exclusivo Treinador dos sub-23 do Leixões como...,Exclusivo Treinador dos sub-23 do Leixões como...


In [9]:
# drop the rows with nan values

nan_rows = df_train.loc[pd.isna(df_train["All"]), :].index
nan_rows
# df_train = df_train.drop([df_train.index[nan_rows[0]], df_train.index[nan_rows[1]]])
# nan_rows = df_train.loc[pd.isna(df_train["All"]), :].index

Int64Index([], dtype='int64')

In [10]:
# some stats

from statistics import mean

def data_stats(df):
    l = list(df.apply(len))
    print("Max lentgh:", max(l))
    print(df.iloc[l.index(max(l))])
    print("\nSmallest length:", min(l))
    print(df.iloc[l.index(min(l))])
    print("\nAvg length:", mean(l))

In [11]:
data_stats(df_train['Context'])

Max lentgh: 671
O juiz Dalton Igor Kita Conrado, da 5ª Vara Federal de Campo Grande, recebeu denúncia realizada pelo MPF (Ministério Público Federal) por peculato contra os médicos Marcelino Chehoud Ibrahim, ex-Diretor Clínico do HU/UFMS; e Amaury Edgardo Mont Serrat Avila Souza, ex-presidente da Comissão Permanente de Licitação do HU/UFMS. A denúncia, que também implica Nilza dos Santos Miranda, Artemisia Mesquita de Almeida, Alcides Manuel do Nascimento e Jorge da Costa Carramanho Junior, é um dos desdobramento da Operação Sangue Frio, da Polícia Federal, que desarticulou esquema de fraudes em licitação e uso indevido de verba pública - também conhecida como “máfia do câncer”.

Smallest length: 72
Eu servi o Exército. Meu sonho era colocar meu filho no Colégio Militar.

Avg length: 278.9622641509434


In [12]:
data_stats(df_train['All'])

Max lentgh: 1025
Vamos lembrar que os pacotes de ajuda nos Estados Unidos durante a crise da Covid-19 já superam os 5 trilhões de dólares nos últimos anos, e até então não conseguimos ver sinais claros de inflação na economia americana como um todo. Ao contrário dos gastos comuns de política fiscal, acompanhamos nos Estados Unidos a compra de títulos públicos e privados por parte do Banco Central, o Federal Reserve, o que significou que muitos dos recursos utilizados pelo Governo para o combate à crise não foram totalmente alocados para os mais pobres, aqueles que consomem mais itens de primeira necessidade, o que causa impacto à inflação de maneira mais direta e severa. Essa situação de compra de títulos, aliada a uma política de juros básicos da economia próximos de zero, fez com que a inflação americana despontasse em outro local, nas ações das empresas, que nunca estiveram com altas tão consistentes, apesar de que a economia global ainda está em processo de reorganização, e o desem

In [13]:
# # split into train and test
# from sklearn.model_selection import train_test_split

# df_train, df_test = train_test_split(df_train, test_size=0.2, random_state=42)

In [14]:
class DataPrep(Dataset):

    def __init__(self, control_code, truncate=False, gpt2_type="facebook/opt-125m", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in df_train['All']:
          self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))

        if truncate:
            self.lyrics = self.lyrics[:20000]
        self.lyrics_count = len(self.lyrics)

    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]

In [15]:
train_dataset = DataPrep(df_train['All'], truncate=True, gpt2_type="facebook/opt-125m")
tokenizer = GPT2Tokenizer.from_pretrained('facebook/opt-125m')

## Fine tuning

In [16]:
# model = OPTForCausalLM.from_pretrained('facebook/opt-125m')
model = OPTForCausalLM.from_pretrained('bigscience/bloom-560m')

You are using a model of type bloom to instantiate a model of type opt. This is not supported for all configurations of models and can yield errors.


Some weights of OPTForCausalLM were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['decoder.layers.1.fc2.bias', 'decoder.layers.7.final_layer_norm.bias', 'decoder.layers.1.self_attn.k_proj.bias', 'decoder.layers.0.self_attn_layer_norm.weight', 'decoder.layers.1.self_attn.k_proj.weight', 'decoder.layers.0.self_attn.k_proj.bias', 'decoder.layers.6.final_layer_norm.bias', 'decoder.layers.6.self_attn_layer_norm.bias', 'decoder.layers.2.fc2.weight', 'decoder.layers.9.self_attn.q_proj.weight', 'decoder.layers.5.fc2.weight', 'decoder.layers.6.self_attn.v_proj.weight', 'decoder.layers.6.fc2.bias', 'decoder.layers.9.self_attn.out_proj.bias', 'decoder.layers.5.fc1.weight', 'decoder.layers.11.self_attn.q_proj.weight', 'decoder.layers.0.self_attn.out_proj.weight', 'decoder.layers.6.fc1.bias', 'decoder.layers.3.self_attn_layer_norm.weight', 'decoder.layers.5.self_attn_layer_norm.bias', 'decoder.layers.0.fc2.bias', 'decoder.embed_tokens.weight', 'decod

In [17]:
#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [18]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=20, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="facebook/opt-125m", output_dir=".", output_prefix="wreckgar",
    test_mode=False, save_model_on_epoch=False,
):

    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [ ]:
#Train the model on the specific data we have
model = train(train_dataset, model, tokenizer)

In [ ]:
#Save the model to a pkl or something so it can be reused later on
torch.save(model, '../Data/few-shot-model-opt-125_pt.pt')

## Text Generation

In [ ]:
# #Load the model to use it
# model = torch.load('../Data/few-shot-model-opt-125_pt.pt')

# no fine tuning
model = OPTForCausalLM.from_pretrained('facebook/opt-125m').cuda()

In [ ]:
import re

# let's remove the newlines and extra spaces
def remove_newlines_and_extra_spaces(string):
    string = string.replace('\n', ' ')
    string = ' '.join(string.split())
    string = re.sub('<[^<]+?>', '', string) # remove html tags
    string = re.sub(r'([.,!?])\1+', r'\1', string)   # remove repeated punctuation
    string = re.sub(r'["]+', '"', string)
    string = re.sub(r'[`]+', '`', string)
    string = re.sub(r"[']+", "'", string)
    return string

In [ ]:
from nltk.tokenize import sent_tokenize

temperature_generator = 0.6

next_generations = []
next_generations_no_t = []

for context in df_test['Context'].iloc[:]:
    all_next = ''
    while len(all_next)<=5:
        inputs = tokenizer.encode(context, return_tensors='pt')
        outputs = model.generate(inputs.cuda(), min_length=len(context)+20, max_length=len(context)+100, do_sample=True, temperature=temperature_generator)
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        all_next = text.replace(context, '')



        all_next = remove_newlines_and_extra_spaces(all_next)

        # if len(all_next) > 4 and bool(re.search('[a-zA-Z]', all_next)):
        if len(all_next) > 4:

            try:
                # get the first sentence only
                text_list = sent_tokenize(all_next)
                next_generations_no_t.append(all_next)

                if len(text_list[0])<len(text_list[1]):
                    next_generations.append(text_list[0]+text_list[1])
                else:
                    next_generations.append(text_list[0])
            except IndexError:
                next_generations.append(all_next)
                if not text:
                    print("...............STRING IS EMPTY...............")
        else:
            all_next = ''
            print(len(next_generations))


In [ ]:
len(next_generations)

238

In [ ]:
df_test['Generated Next'] = next_generations
df_test.shape

(238, 9)

In [ ]:
# save it
df_test.to_csv("../Data/Test Result Data/zero-shot-opt-base-t0_6-pt.csv")

In [ ]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):

    model.eval()

    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False

            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break

            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>"
              generated_list.append(output_text)

    return generated_list

In [ ]:
# Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_lyrics = []
  for i in range(len(test_data)):
    x = generate(model.to('cpu'), tokenizer, test_data['Context'][i], entry_count=1)
    generated_lyrics.append(x)
  return generated_lyrics

In [ ]:
df_test = df_test.reset_index()
df_test.head()

,index,Unnamed: 0,MWE,Previous,Target,Next,Context,All
0,880,1653,big cheese,My name is Heddi.,I'm the big cheese founder of travel gift card...,Her name is Muppet.,My name is Heddi.I'm the big cheese founder of...,My name is Heddi.I'm the big cheese founder of...
1,1468,2846,rock bottom,"McLeish says the Scottish government ""doesn't ...",The relationships with government are rock bot...,What have the lower leagues done to create thi...,"McLeish says the Scottish government ""doesn't ...","McLeish says the Scottish government ""doesn't ..."
2,1859,152,panda car,"n 1967, the Dunbartonshire force bought two Hi...","The boot lids, bonnets and doors were then swa...","In the 1980s, police cars in the United Kingdo...","n 1967, the Dunbartonshire force bought two Hi...","n 1967, the Dunbartonshire force bought two Hi..."
3,1379,2673,role model,Hunt moved out of home to the Gold Coast to se...,He had started his first business at 14 and si...,Hunt is now the acting CEO of the Hunt Retail ...,Hunt moved out of home to the Gold Coast to se...,Hunt moved out of home to the Gold Coast to se...
4,600,1095,agony aunt,"Let's kick off with the original virgin.""",He was joined by long-standing presenting part...,Have a problem you're willing to share with th...,"Let's kick off with the original virgin.""He wa...","Let's kick off with the original virgin.""He wa..."


In [ ]:
generated_lyrics = text_generation(df_test)

100%|██████████| 1/1 [00:08<00:00,  8.79s/it]


In [ ]:
#Loop to keep only generated text and add it as a new column in the dataframe
my_generations=[]

for i in range(len(generated_lyrics)):
  a = df_test['Context'][i].split()[-30:] #Get the matching string we want (30 words)
  b = ' '.join(a)
  c = ' '.join(generated_lyrics[i]) #Get all that comes after the matching string
  my_generations.append(c.split(b)[-1])

df_test['Generated Next'] = my_generations

In [ ]:
#Finish the sentences when there is a point, remove after that
final=[]

for i in range(len(df_test)):
    if '.' in df_test['Generated Next'][i]:
        to_remove = df_test['Generated Next'][i].split('.')[-1]
        final.append(df_test['Generated Next'][i].replace(to_remove,''))
    else:
        final.append(df_test['Generated Next'][i])


df_test['Generated Next'] = final
df_test['Generated Next'] = df_test['Generated Next'].str.replace(r'<\|endoftext\|>', '')

df_test.head()

<ipython-input-38-121d4e469d22>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['Generated Next'] = df_test['Generated Next'].str.replace(r'<\|endoftext\|>', '')


,index,Unnamed: 0,MWE,Previous,Target,Next,Context,All,Generated Next
0,880,1653,big cheese,My name is Heddi.,I'm the big cheese founder of travel gift card...,Her name is Muppet.,My name is Heddi.I'm the big cheese founder of...,My name is Heddi.I'm the big cheese founder of...,</s>My name is Heddi.I'm the big cheese founde...
1,1468,2846,rock bottom,"McLeish says the Scottish government ""doesn't ...",The relationships with government are rock bot...,What have the lower leagues done to create thi...,"McLeish says the Scottish government ""doesn't ...","McLeish says the Scottish government ""doesn't ...","""The Government doesn't trust it either.""He al..."
2,1859,152,panda car,"n 1967, the Dunbartonshire force bought two Hi...","The boot lids, bonnets and doors were then swa...","In the 1980s, police cars in the United Kingdo...","n 1967, the Dunbartonshire force bought two Hi...","n 1967, the Dunbartonshire force bought two Hi...","Red with one cross and one half door, the othe..."
3,1379,2673,role model,Hunt moved out of home to the Gold Coast to se...,He had started his first business at 14 and si...,Hunt is now the acting CEO of the Hunt Retail ...,Hunt moved out of home to the Gold Coast to se...,Hunt moved out of home to the Gold Coast to se...,He said it would be “truly fitting” for him to...
4,600,1095,agony aunt,"Let's kick off with the original virgin.""",He was joined by long-standing presenting part...,Have a problem you're willing to share with th...,"Let's kick off with the original virgin.""He wa...","Let's kick off with the original virgin.""He wa...",Mr Kennedy's latest appearance came in a touch...


In [ ]:
# save it
df_test.to_csv("../Data/Test Result Data/generated_opt-125.csv")